In [1]:
import sys
import numpy as np
from scipy.special import softmax
import json
import torch
from torch import onnx
import torch.nn.functional as F
import onnxruntime as ort
from insilico_conv import InSilicoConv
from insilico_lrcn import InSilicoLRCN
from insilico_lstm import InSilicoLSTM
from captum.attr import KernelShap, GradientShap
from SpecReX.wrapper import SReX


import shap
from functools import partial
import matplotlib.pyplot as plt

torch.backends.cudnn.enabled=False


ModuleNotFoundError: No module named 'insilico_conv'

In [3]:
#For testing 
single_peak_0 = np.load('../Simulated Spectral Dataset/Mean Spectra/single_peak_class_0_mean.npy')
single_peak_1 = np.load('../Simulated Spectral Dataset/Mean Spectra/single_peak_class_1_mean.npy')
single_peak_2 = np.load('../Simulated Spectral Dataset/Mean Spectra/single_peak_class_2_mean.npy')

double_peak_0 = np.load('../Simulated Spectral Dataset/Mean Spectra/double_peak_class_0_mean.npy')
double_peak_1 = np.load('../Simulated Spectral Dataset/Mean Spectra/double_peak_class_1_mean.npy')
double_peak_2 = np.load('../Simulated Spectral Dataset/Mean Spectra/double_peak_class_2_mean.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../Simulated Spectral Dataset/Mean Spectra/single_peak_class_0_mean.npy'

In [4]:
#For Background
background_class_0_single = np.load('../Simulated Spectral Dataset/Single Peak/Train/train20260_class0.npy')
background_class_1_single = np.load('../Simulated Spectral Dataset/Single Peak/Train/train3155_class1.npy')
background_class_2_single = np.load('../Simulated Spectral Dataset/Single Peak/Train/train42360_class2.npy')

backgroud_class_0_double = np.load('../Simulated Spectral Dataset/Double Peak/Train/train3157_class0.npy')
background_class_1_double = np.load('../Simulated Spectral Dataset/Double Peak/Train/train53175_class1.npy')
background_class_2_double = np.load('../Simulated Spectral Dataset/Double Peak/Train/train4237_class2.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../Simulated Spectral Dataset/Single Peak/Train/train20260_class0.npy'

In [ ]:
def f(x):
    x = x.reshape(1,1,-1).astype(np.float32)
    return softmax(np.array(ort_sess.run(None, {'input': x}))).reshape(1,-1)

In [ ]:
def f(x):
    x = torch.from_numpy(x.reshape(1,1,-1).astype(np.float32)).cuda()
    return softmax(model(x).detach().cpu().numpy())

In [ ]:
def ReLU(x):
    return x * (x > 0)

In [ ]:
def current_non_zero(x):
    current_val = 0
    for i in range(len(x)):
        if x[i] != 0:
            current_val = x[i]
        else:
            x[i] = current_val
    return x

### Single Peak Conv

In [3]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Single Peak Conv Null/tune_with_parameters_2024-08-19_13-25-03/tune_with_parameters_21b89_00001_1_dp=0.2000,k1=7,k2=1,k3=3,k4=1,lr=0.0426_2024-08-19_13-25-34/checkpoint_000002/checkpoint.pt')

#Load Model
model = InSilicoConv(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.1,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

# Creating a representative input
batch_size = 32
x = torch.rand(batch_size,1,852, device='cuda:0')

# Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/single_peak_conv.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

/users/k23058970/miniforge3/envs/Training/lib/python3.10/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [4]:
srex = SReX(model)

In [5]:
resp = srex.calc_responsibility(spectra = single_peak_0.reshape(1,-1), wn = np.arange(len(single_peak_0)).reshape(1,-1))

Spectra Classified as %s [2]


  0%|          | 0/50 [00:00<?, ?it/s]

random seed = %d 42


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [8]:
len(single_peak_0)

852

In [ ]:
model_state['conv4.weight'].shape[0]

In [ ]:
model_state['conv4.weight'].shape

In [ ]:
# Test PyTorch model
model.eval()
with torch.no_grad():
    print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float64)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float64)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float64)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
model = partial(model,f)

In [ ]:
%%time
ks = KernelShap(model)
attr_ks = ks.attribute(torch.from_numpy(single_peak_0.reshape(1,1,-1).astype(np.float32)).cuda(), target = 0, n_samples=10000).reshape(-1)

In [ ]:
%%time
gs = GradientShap(model)
attr_gs = gs.attribute(torch.from_numpy(single_peak_2.reshape(1,1,-1).astype(np.float32)).cuda(), torch.zeros(1,1,852).cuda(), target = 2, n_samples = 30000)

In [ ]:
fig, axs = plt.subplots(nrows=1,
                       ncols=1,
                       figsize=(15,6))

shap_vals = ReLU(attr_ks.detach().cpu().numpy().reshape(-1))

axs.plot(np.abs(shap_vals) / np.max(np.abs(shap_vals)))
fig.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_vals) / np.max(np.abs(shap_vals))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_vals):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_2, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))


explainer_class_0 = shap.DeepExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.DeepExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.DeepExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(single_peak_0.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(single_peak_1.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(single_peak_2.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
single_peak_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2,
                       ncols=1,
                       figsize=(15,12))

axs[0].plot(ReLU(shap_values_class_0))
fig.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1,
                       ncols=1,
                       figsize=(12,6))

magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

axs.plot(magnitude)
fig.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_2, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2,
                       ncols=1,
                       figsize=(15,12))

axs[0].plot(ReLU(shap_values_class_2))
fig.show()

In [ ]:
np.argmax(shap_values_class_2)

### Single Peak LRCN

In [4]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Single Peak LRCN Null/tune_with_parameters_2024-08-19_13-29-31/tune_with_parameters_c1630_00004_4_dp=0.2500,k1=1,k2=5,k3=1,k4=5,lr=0.0873_2024-08-19_13-30-01/checkpoint_000001/checkpoint.pt')

#Load Model
model = InSilicoLRCN(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.2,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,852, device='cuda:0')

#Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/single_peak_lrcn.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

/users/k23058970/miniforge3/envs/Training/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4661: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [ ]:
model_state['conv1.weight'].shape

In [ ]:
#Test PyTorch model
with torch.no_grad():
    print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#SHAP Values
ks = KernelShap(model)
attr_ks = ks.attribute(torch.from_numpy(single_peak_0.reshape(1,1,-1).astype(np.float32)).cuda(), target = 0, n_samples=2000, show_progress = True).reshape(-1)
shap_vals = ReLU(attr_ks.detach().cpu().numpy())

In [ ]:
fig, axs = plt.subplots(nrows=2,
                       ncols=1,
                       figsize=(15,12))

# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_vals) / np.max(np.abs(shap_vals))

axs[0].plot(shap_vals, color ='black')
axs[0].set_title('Saliency Landscape')
axs[0].set_xlabel('Wavenumber')
axs[0].set_ylabel('Intensity (A.U.)')

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_vals):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    axs[1].axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
axs[1].plot(single_peak_0, label='Class 0 Mean', color='black', linewidth=1.5)

# Enhance the plot
axs[1].set_title('SHAP Values and Spectrum for Class 0')
axs[1].set_xlabel('Wavenumber')
axs[1].set_ylabel('Intensity (A.U.)')
axs[1].grid(True)  # Optionally add grid for better visual tracking of features

fig.savefig('SHAP Saliency Landscape CNN class 0.png',
        dpi = 900,
        bbox_inches = "tight")

In [ ]:


# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_vals):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_2, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_lrcn.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))

model.train()

explainer_class_0 = shap.KernelExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.KernelExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.KernelExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(single_peak_0.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(single_peak_1.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(single_peak_2.reshape(1,1,-1).astype(np.float32)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_2, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

### Single Peak LSTM

In [5]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Single Peak LSTM Null/tune_with_parameters_2024-08-19_13-32-44/tune_with_parameters_3497d_00000_0_dp=0.2000,hidden_size=400,lr=0.0022,num_layers=1_2024-08-19_13-33-15/checkpoint_000021/checkpoint.pt')

# #Load Model
model = InSilicoLSTM(
    hidden_size=400,
    num_layers=1,
    dp=0.1,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

# #Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,852, device='cuda:0')

# Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/single_peak_lstm.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
with torch.no_grad():
    print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_lstm.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))

model.train()

explainer_class_0 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(single_peak_0.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(single_peak_1.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(single_peak_2.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(single_peak_1, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

### Double Peak Conv

In [7]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Double Peak Conv Null/tune_with_parameters_347d1_00000_0_dp=0.2500,k1=1,k2=5,k3=5,k4=5,lr=0.0525_2024-08-19_21-54-19/checkpoint_000007/checkpoint.pt')

#Load Model
model = InSilicoConv(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.1,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,852, device='cuda:0')

#Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/double_peak_conv_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
with torch.no_grad():
    print('Class 0:',F.softmax(model(torch.Tensor(double_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 1:',F.softmax(model(torch.Tensor(double_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 2:',F.softmax(model(torch.Tensor(double_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/double_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': double_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': double_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': double_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))


explainer_class_0 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(double_peak_0.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(double_peak_1.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(double_peak_2.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_2, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

### Double Peak LRCN

In [8]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Double Peak LRCN Null/tune_with_parameters_54ae7_00002_2_dp=0.2500,k1=5,k2=3,k3=3,k4=1,lr=0.0141_2024-08-19_21-55-13/checkpoint_000018/checkpoint.pt')

#Load Model
model = InSilicoLRCN(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.2,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,852, device='cuda:0')

#Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/double_peak_lrcn_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
with torch.no_grad():
    print('Class 0:',F.softmax(model(torch.Tensor(double_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 1:',F.softmax(model(torch.Tensor(double_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
    print('Class 2:',F.softmax(model(torch.Tensor(double_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/double_peak_lrcn.onnx')
output_class_0 = ort_sess.run(None, {'input': double_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': double_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': double_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))


explainer_class_0 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(double_peak_0.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(double_peak_1.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(double_peak_2.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_2, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

### Double Peak LSTM

In [10]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Double Peak LSTM Null/tune_with_parameters_34c49_00004_4_dp=0.1000,hidden_size=500,lr=0.0013,num_layers=1_2024-08-19_22-08-39/checkpoint_000025/checkpoint.pt')

# #Load Model
model = InSilicoLSTM(
    hidden_size=500,
    num_layers=1,
    dp=0.1,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,852)
)

model.load_state_dict(model_state)
model.to(device)

# #Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,852, device='cuda:0')

# Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/double_peak_lstm_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
#SHAP Values
ks = KernelShap(model)
attr_ks = ks.attribute(torch.from_numpy(double_peak_2.reshape(1,1,-1).astype(np.float32)).cuda(), target = 2, n_samples=2000, show_progress = True).reshape(-1)
shap_vals = ReLU(attr_ks.detach().cpu().numpy())

In [ ]:
fig, axs = plt.subplots(nrows=2,
                       ncols=1,
                       figsize=(15,12))

# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_vals) / np.max(np.abs(shap_vals))

axs[0].plot(shap_vals, color ='black')
axs[0].set_title('Saliency Landscape')
axs[0].set_xlabel('Wavenumber')
axs[0].set_ylabel('Intensity (A.U.)')

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_vals):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    axs[1].axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
axs[1].plot(double_peak_2.reshape(-1), label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
axs[1].set_title('SHAP Values and Spectrum for Class 2')
axs[1].set_xlabel('Wavenumber')
axs[1].set_ylabel('Intensity (A.U.)')
axs[1].grid(True)  # Optionally add grid for better visual tracking of features

fig.savefig('SHAP Saliency Landscape LSTM class 2.png',
        dpi = 900,
        bbox_inches = "tight")

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
print('Class 0:',F.softmax(model(torch.Tensor(double_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 1:',F.softmax(model(torch.Tensor(double_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 2:',F.softmax(model(torch.Tensor(double_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': double_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': double_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': double_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

In [ ]:
#For calculating SHAP

# # Sample background data from the reshaped tensor
background_indices = np.random.choice(single_peak_0.shape[0], 852, replace=True)
background = np.zeros(shape = (1,852))


explainer_class_0 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_1 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())
explainer_class_2 = shap.GradientExplainer(model, torch.from_numpy(background.reshape(1,1,-1).astype(np.float32)).cuda())

shap_values_class_0 = explainer_class_0.shap_values(torch.from_numpy(double_peak_0.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,0]
shap_values_class_1 = explainer_class_1.shap_values(torch.from_numpy(double_peak_1.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,1]
shap_values_class_2 = explainer_class_2.shap_values(torch.from_numpy(double_peak_2.reshape(1,1,-1)).cuda()).reshape(852,-1)[:,2]

shap_values_class_0.shape, shap_values_class_1.shape, shap_values_class_2.shape

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_0) / np.max(np.abs(shap_values_class_0))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_0):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_0, label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 1')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_1) / np.max(np.abs(shap_values_class_1))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_1):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_1, label='Class 2 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 2')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

In [ ]:
# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_values_class_2) / np.max(np.abs(shap_values_class_2))

# Set up the plot
plt.figure(figsize=(12, 6))

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_values_class_2):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    plt.axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
plt.plot(double_peak_2, label='Class 3 Mean', color='black', linewidth=1.5)

# Enhance the plot
plt.title('SHAP Values and Spectrum for Class 3')
plt.xlabel('Wavenumber shift')
plt.ylabel('Intensity (A.U.)')
plt.legend()
plt.grid(True)  # Optionally add grid for better visual tracking of features
plt.show()

### Complex Peak Cov

In [11]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Complex Peak Conv Null/tune_with_parameters_b687f_00000_0_dp=0.1000,k1=1,k2=5,k3=3,k4=5,lr=0.0160_2024-08-19_22-55-13/checkpoint_000001/checkpoint.pt')

#Load Model
model = InSilicoConv(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.1,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,1752)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,1752, device='cuda:0')

#Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/complex_peak_conv_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

### Complex Peak LRCN

In [12]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#Load config
model_state, _ = torch.load('results/Complex Peak LRCN Null/tune_with_parameters_a4509_00001_1_dp=0.2000,k1=5,k2=1,k3=5,k4=5,lr=0.0679_2024-08-19_22-54-43/checkpoint_000006/checkpoint.pt')

#Load Model
model = InSilicoLRCN(
    nc1=model_state['conv1.weight'].shape[0],
    k1=model_state['conv1.weight'].shape[-1],
    nc2=model_state['conv2.weight'].shape[0],
    k2=model_state['conv2.weight'].shape[-1],
    nc3=model_state['conv3.weight'].shape[0],
    k3=model_state['conv3.weight'].shape[-1],
    nc4=model_state['conv4.weight'].shape[0],
    k4=model_state['conv4.weight'].shape[-1],
    dp=0.25,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,1752)
)

model.load_state_dict(model_state)
model.to(device)

#Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,1752, device='cuda:0')

#Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/complex_peak_lrcn_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
#Test PyTorch model
print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))

### Complex Peak LSTM

In [14]:
#Convert Model to ONNX
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

#F1-score: 0.84
#Load config
model_state, _ = torch.load('results/Complex Peak LSTM Null/tune_with_parameters_cf806_00007_7_dp=0.2000,hidden_size=200,lr=0.0462,num_layers=1_2024-08-19_22-55-55/checkpoint_000006/checkpoint.pt')

# #Load Model
model = InSilicoLSTM(
    hidden_size=200,
    num_layers=1,
    dp=0.2,
    l1=model_state['fc1.weight'].shape[0],
    l2=model_state['fc2.weight'].shape[0],
    init_input=torch.rand(1,1,1752)
)

model.load_state_dict(model_state)
model.to(device)

# #Put Model in inference mode
model.eval()

#Creating a representative input
batch_size = 1
x = torch.rand(batch_size,1,1752, device='cuda:0')

# Model to onnx
torch.onnx.export(
    model,
    x,
    'Final Models/complex_peak_lstm_null.onnx',
    export_params=True,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input':{0:'batch_size'},
                  'output':{0:'batch_size'}}
)

In [ ]:
model_state['fc2.weight'].shape[0]

In [ ]:
complex_peak_class_1 = np.load("../ComplexPeakClass_1 Mean 1.npy")

In [ ]:
#SHAP Values
ks = KernelShap(model)
attr_ks = ks.attribute(torch.from_numpy(complex_peak_class_1.reshape(1,1,-1).astype(np.float32)).cuda(), target = 2, n_samples=2000, show_progress = True).reshape(-1)
shap_vals = ReLU(attr_ks.detach().cpu().numpy())

In [ ]:
fig, axs = plt.subplots(nrows=2,
                       ncols=1,
                       figsize=(15,12))

# Normalize the magnitude of SHAP values to determine the color intensity
magnitude = np.abs(shap_vals) / np.max(np.abs(shap_vals))

axs[0].plot(shap_vals, color ='black')
axs[0].set_title('Saliency Landscape')
axs[0].set_xlabel('Wavenumber')
axs[0].set_ylabel('Intensity (A.U.)')

# Plot SHAP values as colored vertical lines across the entire y-axis
for i, value in enumerate(shap_vals):
    color = 'red' if value > 0 else 'blue'
    alpha = magnitude[i]
    axs[1].axvline(x=i, color=color, alpha=alpha, linewidth=2)

# Overlay the test sample spectrum as a line plot
axs[1].plot(complex_peak_class_1.reshape(-1), label='Class 1 Mean', color='black', linewidth=1.5)

# Enhance the plot
axs[1].set_title('SHAP Values and Spectrum for Class 1')
axs[1].set_xlabel('Wavenumber')
axs[1].set_ylabel('Intensity (A.U.)')
axs[1].grid(True)  # Optionally add grid for better visual tracking of features

fig.savefig('SHAP Saliency Landscape LSTM class 1.png',
        dpi = 900,
        bbox_inches = "tight")

In [ ]:
#Test PyTorch model
print('Class 0:',F.softmax(model(torch.Tensor(single_peak_0.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 1:',F.softmax(model(torch.Tensor(single_peak_1.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))
print('Class 2:',F.softmax(model(torch.Tensor(single_peak_2.astype(np.float32)).to('cuda').reshape(1,1,-1)),dim=-1))

In [ ]:
#Test ONNX model 
ort_sess = ort.InferenceSession('Final Models/single_peak_conv.onnx')
output_class_0 = ort_sess.run(None, {'input': single_peak_0.reshape(1,1,-1).astype(np.float32)})
output_class_1 = ort_sess.run(None, {'input': single_peak_1.reshape(1,1,-1).astype(np.float32)})
output_class_2 = ort_sess.run(None, {'input': single_peak_2.reshape(1,1,-1).astype(np.float32)})

print('Class 0 ONNX:',softmax(output_class_0, axis=-1))
print('Class 1 ONNX:',softmax(output_class_1, axis=-1))
print('Class 2 ONNX:',softmax(output_class_2, axis=-1))